# PyIceberg 🐍 + duckdb 🦆 Guide
Hey, welcome! 

This guide gets you started with DuckDB and PyIceberg on Tabular.

### Installation:
- clone this repo (just do it, it takes 2 seconds)
- cd into this folder
- install dependencies: 
  - `pip install "pyiceberg[duckdb]" jupyterlab pytz`
  - OR, If you use pipenv like I do (`pip install pipenv`), you can just run `pipenv install` and it'll handle everything

See, that wasn't so bad.

### Tabular Requirements:
- head over to app.tabular.io and log in (or signup if you don't already have an account)
- go to connections > security > service account and hit the big + button to create a new credential
- assign your service account credential to a role that has the correct access for what you want to do (if you don't know, `EVERYONE` is a pretty safe default)
- copy that credential!
- come back here and create a `.env` file in this directory (`guides/pyiceberg_duckdb/.env`). Edit it to look like below and make sure to SAVE IT.
```
TABULAR_CREDENTIAL=t-asdf:1234
```
⬆️ replace `t-asdf:1234` with the tabular credential you just created. 

Good job! Now we're ready to get down to business 💪

### Starting Jupyter Lab:
- Seriously, make sure you save that env file. 
- pipenv users can just run `pipenv run jupyter lab` to fire up jupyter lab. pipenv will load up your credential for you and all will be well
- if this is scary, you can ignore the `.env` file and just paste your credential in plaintext directly in this notebook -- but you should feel bad about your craftsmanship.


*One last note* -- you definitely don't have the weather data that I have in your warehouse. It's just an example. Connect to whatever data you want to.

Happy building! 🧊

In [1]:
import os

from pyiceberg.catalog import load_catalog

# You'll need a tabular credential. Member credential or service account will work fine
TABULAR_CREDENTIAL       = os.environ['TABULAR_CREDENTIAL']
TABULAR_TARGET_WAREHOUSE = 'enterprise_data_warehouse' # replace this with your tabular warehouse name
TABULAR_CATALOG_URI      = 'https://api.tabular.io/ws' # unless you're a single tenant user, you don't need to change this

catalog_properties = {
    'uri':        TABULAR_CATALOG_URI,
    'credential': TABULAR_CREDENTIAL,
    'warehouse':  TABULAR_TARGET_WAREHOUSE
}
catalog = load_catalog(**catalog_properties)

In [11]:
# load the weather data 🌞
tbl = catalog.load_table("batch_raw.serverless_weather_raw")

# query with duckdb
con = tbl.scan().to_duckdb(table_name="serverless_weather_raw")
res = con.execute("""
    select 
        name as city_name,
        avg(main.temp) as avg_temp_degrees_f
    
    from serverless_weather_raw 

    where to_timestamp(dt) > NOW() - INTERVAL '4' HOUR

    group by city_name
""").fetchall()


print('Duckdb 🦆 average temp by city over the last few hours:')
for row in res:
    print(row)

Duckdb 🦆 average temp by city over the last few hours:
('New York', 49.662666666666674)
('Indianapolis', 59.60021739130434)
('London', 53.29913043478261)
('San Jose', 58.48326086956522)
('Nashville', 73.93522727272726)
('Tokyo', 51.41891304347825)
('St Louis', 74.50222222222222)
('Paris', 55.035333333333334)
('Sydney', 67.05717391304348)


In [22]:
# single row retrieval -- get latest temp by region
res = con.execute("""
    with numbered_rows AS (
      select 
        name as city_name,
        main.temp as temp_degrees_f,
        to_timestamp(dt) as temp_measured_at,
        row_number() over (partition by city_name order by dt desc) as row_num
        
      from
        serverless_weather_raw
    )
    
    select
        city_name,
        temp_degrees_f,
        strftime(temp_measured_at, '%Y-%m-%d %H:%M:%S') 
        
    from 
        numbered_rows
        
    where 
        row_num = 1
""").fetchall()


print('Duckdb 🦆 latest temp by city:')
for row in res:
    print(row)

Duckdb 🦆 latest temp by city over the last few hours:
('Paris', 52.36, '2024-04-01 15:07:17')
('Tokyo', 49.98, '2024-04-01 15:07:08')
('Sydney', 66.36, '2024-04-01 15:03:42')
('St Louis', 78.06, '2024-04-01 15:09:55')
('Nashville', 75.83, '2024-04-01 15:03:33')
('London', 50.94, '2024-04-01 15:04:51')
('New York', 50.65, '2024-04-01 15:08:09')
('Indianapolis', 62.94, '2024-04-01 15:00:36')
('San Jose', 65.79, '2024-04-01 15:03:09')
